In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Project/'
%pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project


'/content/drive/.shortcut-targets-by-id/1XSIbHWFizvrGifGPTwtsmagK2AS8ZE7n/Project'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
np.random.seed(682)
tf.random.set_seed(682)

In [ ]:
class DistillerT1S2(keras.Model):

    def __init__(self, student1, student2, teacher):
        super(DistillerT1S2, self).__init__()
        self.teacher = teacher
        self.student1 = student1
        self.student2 = student2

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(DistillerT1S2, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape_student_1:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=True)
            student_predictions_2 = self.student2(x, training=False)

            # Compute losses
            student_loss_1 = self.student_loss_fn(y, student_predictions_1)
            student_loss_2 = self.student_loss_fn(y, student_predictions_2)

            # for student 1
            distillation_loss_t_s1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1)
            )
            distillation_loss_s1_s2 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1)
            )

            loss_1 = 0.1 * student_loss_1 + 0.8 * distillation_loss_t_s1 + 0.1 * (distillation_loss_s1_s2)

        trainable_vars_1 = self.student1.trainable_variables
        gradients_1 = tape_student_1.gradient(loss_1, trainable_vars_1)

        self.optimizer.apply_gradients(zip(gradients_1, trainable_vars_1))

        self.compiled_metrics._metrics[0].update_state(y, student_predictions_1)
        

        with tf.GradientTape() as tape_student_2:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=False)
            student_predictions_2 = self.student2(x, training=True)

            # Compute losses
            student_loss_1 = self.student_loss_fn(y, student_predictions_1)
            student_loss_2 = self.student_loss_fn(y, student_predictions_2)
            
            # for student 2
            distillation_loss_t_s2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1)
            )
            distillation_loss_s2_s1 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1)
            )

            loss_2 = 0.1 * student_loss_2 + 0.8 * distillation_loss_t_s2 + 0.1 * (distillation_loss_s2_s1)

        # Compute gradients
        trainable_vars_2 = self.student2.trainable_variables
        gradients_2 = tape_student_2.gradient(loss_2, trainable_vars_2)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients_2, trainable_vars_2))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics._metrics[1].update_state(y, student_predictions_2)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update (
            {"student_loss_1": student_loss_1,
             "student_loss_2": student_loss_2,
             "distillation_loss_t_s1": distillation_loss_t_s1,
             "distillation_loss_s1_s2": distillation_loss_s1_s2,
             "distillation_loss_t_s2": distillation_loss_t_s2,
             "distillation_loss_s2_s1": distillation_loss_s2_s1}
        )

        return results


    def test_step(self, data):
        
        # Unpack the data
        x, y = data

        # Compute predictions
        student_predictions_1 = self.student1(x, training=False)
        student_predictions_2 = self.student2(x, training=False)

        # Calculate the loss
        student_loss_1 = self.student_loss_fn(y, student_predictions_1)
        student_loss_2 = self.student_loss_fn(y, student_predictions_2)

        # Update the metrics.
        self.compiled_metrics._metrics[0].update_state(y, student_predictions_1)
        self.compiled_metrics._metrics[1].update_state(y, student_predictions_2)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update({"student_loss_1": student_loss_1,
             "student_loss_2": student_loss_2})

        return results


In [ ]:
class DistillerT3S1(keras.Model):

    def __init__(self, student1, student2, student3, teacher):
        super(DistillerT3S1, self).__init__()
        self.teacher = teacher
        self.student1 = student1
        self.student2 = student2
        self.student3 = student3

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(DistillerT3S1, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape_student_3:
            
            # Forward pass of student
            student_predictions_1 = self.student1(x, training=False)
            student_predictions_2 = self.student2(x, training=False)
            student_predictions_3 = self.student3(x, training=True)

            # Compute losses
            student_loss_3 = self.student_loss_fn(y, student_predictions_3)
            
            # for student 3
            distillation_loss_t_s3 = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )
            distillation_loss_s1_s3 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_1 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )
            distillation_loss_s2_s3 = self.distillation_loss_fn(
                tf.nn.softmax(student_predictions_2 / self.temperature, axis=1),
                tf.nn.softmax(student_predictions_3 / self.temperature, axis=1)
            )

            loss_3 = 0.1 * student_loss_3 + 0.6 * distillation_loss_t_s3 + 0.3 * (distillation_loss_s1_s3 + distillation_loss_s2_s3)

        # Compute gradients
        trainable_vars_3 = self.student3.trainable_variables
        gradients_3 = tape_student_3.gradient(loss_3, trainable_vars_3)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients_3, trainable_vars_3))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics._metrics[0].update_state(y, student_predictions_3)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update (
            {"student_loss_3": student_loss_3,
             "distillation_loss_t_s3": distillation_loss_t_s3,
             "distillation_loss_s1_s3": distillation_loss_s1_s3,
             "distillation_loss_s2_s3": distillation_loss_s2_s3}
        )

        return results


    def test_step(self, data):
        
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction_3 = self.student3(x, training=False)

        # Calculate the loss
        student_loss_3 = self.student_loss_fn(y, y_prediction_3)

        # Update the metrics.
        self.compiled_metrics._metrics[0].update_state(y, y_prediction_3)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.compiled_metrics._metrics}

        results.update({"student_loss_3": student_loss_3})

        return results


In [ ]:
teacher = tf.keras.models.load_model('/saved_models/resnet50cifar')

In [ ]:
# Student 3

def get_student3():

  def preprocess_image_input(input_images):
    input_images = tf.cast(input_images, 'float32')
    output_ims = tf.keras.applications.mobilenet.preprocess_input(input_images)
    return output_ims

  class Preprocess(tf.keras.layers.Layer):
      def __init__(self):
          super(Preprocess, self).__init__()

      def call(self, inputs):
          return preprocess_image_input(inputs)

  student_mobile = tf.keras.applications.MobileNet(
      input_shape=(224, 224, 3),
      alpha=1.0,
      depth_multiplier=1,
      dropout=0.001,
      include_top=True,
      weights=None,
      input_tensor=None,
      pooling=None,
      classes=10,
      classifier_activation=None
  )

  inputs = tf.keras.layers.Input(shape=(32,32,3))
  resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
  pre_process = Preprocess()(resize)
  resnet_extractor = student_mobile(pre_process)
  student = tf.keras.Model(inputs=inputs, outputs = resnet_extractor)
  return student

student3 = get_student3()

In [ ]:
# Student 2

def get_student2():

	# Import necessary components to build LeNet
	from keras.models import Sequential
	from keras.layers.core import Dense, Activation, Flatten
	from keras.layers.convolutional import Conv2D, MaxPooling2D
	from keras.regularizers import l2

	def lenet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0.,
		weights=None):

		# Initialize model
		lenet = Sequential()

		# 2 sets of CRP (Convolution, RELU, Pooling)
		lenet.add(Conv2D(20, (5, 5), padding="same",
			input_shape=img_shape, kernel_regularizer=l2(l2_reg)))
		lenet.add(Activation("relu"))
		lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		lenet.add(Conv2D(50, (5, 5), padding="same",
			kernel_regularizer=l2(l2_reg)))
		lenet.add(Activation("relu"))
		lenet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# Fully connected layers (w/ RELU)
		lenet.add(Flatten())
		lenet.add(Dense(500, kernel_regularizer=l2(l2_reg)))
		lenet.add(Activation("relu"))

		# Softmax (for classification)
		lenet.add(Dense(n_classes, kernel_regularizer=l2(l2_reg)))
		# lenet.add(Activation("softmax"))

		if weights is not None:
			lenet.load_weights(weights)

		# Return the constructed network
		return lenet

	# Create the student
	student = lenet_model()
	return student
	
student2 = get_student2()

In [ ]:
# Student 1

def get_student1():

	from keras.models import Sequential
	from keras.layers.core import Dense, Dropout, Activation, Flatten
	from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
	from keras.layers import BatchNormalization
	from keras.regularizers import l2

	def alexnet_model(img_shape=(32, 32, 3), n_classes=10, l2_reg=0., weights=None):

		# Initialize model
		alexnet = Sequential()

		# Layer 1
		alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
			padding='same', kernel_regularizer=l2(l2_reg)))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(MaxPooling2D(pool_size=(2, 2)))

		# Layer 2
		alexnet.add(Conv2D(256, (5, 5), padding='same'))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(MaxPooling2D(pool_size=(2, 2)))

		# Layer 3
		alexnet.add(ZeroPadding2D((1, 1)))
		alexnet.add(Conv2D(512, (3, 3), padding='same'))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(MaxPooling2D(pool_size=(2, 2)))

		# Layer 4
		alexnet.add(ZeroPadding2D((1, 1)))
		alexnet.add(Conv2D(1024, (3, 3), padding='same'))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))

		# Layer 5
		alexnet.add(ZeroPadding2D((1, 1)))
		alexnet.add(Conv2D(1024, (3, 3), padding='same'))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(MaxPooling2D(pool_size=(2, 2)))

		# Layer 6
		alexnet.add(Flatten())
		alexnet.add(Dense(3072))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(Dropout(0.5))

		# Layer 7
		alexnet.add(Dense(4096))
		alexnet.add(BatchNormalization())
		alexnet.add(Activation('relu'))
		alexnet.add(Dropout(0.5))

		# Layer 8
		alexnet.add(Dense(n_classes))

		if weights is not None:
			alexnet.load_weights(weights)

		return alexnet

	# Create the student
	student = alexnet_model()
	return student

student1 = get_student1()

In [ ]:
# Prepare the train and test dataset.
batch_size = 64

(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 7s 0us/step


In [ ]:
# Initialize and compile distiller

# Teach S1 and S2 from T
student1 = get_student1()
student2 = get_student2()
student3 = get_student3()
distiller1 = DistillerT1S2(student1=student1, student2=student2, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')

distiller1.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller1.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller1.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 174s 103ms/step - s1_acc: 0.2902 - s2_acc: 0.1519 - student_loss_1: 87.7904 - student_loss_2: 2.4689 - distillation_loss_t_s1: 0.0207 - distillation_loss_s1_s2: 0.0375 - distillation_loss_t_s2: 0.0213 - distillation_loss_s2_s1: 0.6856
Epoch 2/5
1563/1563 [==============================] - 160s 103ms/step - s1_acc: 0.4112 - s2_acc: 0.2620 - student_loss_1: 2.3173 - student_loss_2: 1.5926 - distillation_loss_t_s1: 0.0210 - distillation_loss_s1_s2: 0.0164 - distillation_loss_t_s2: 0.0081 - distillation_loss_s2_s1: 0.0678
Epoch 3/5
1563/1563 [==============================] - 160s 102ms/step - s1_acc: 0.4778 - s2_acc: 0.3542 - student_loss_1: 2.0717 - student_loss_2: 1.3483 - distillation_loss_t_s1: 0.0188 - distillation_loss_s1_s2: 0.0147 - distillation_loss_t_s2: 0.0102 - distillation_loss_s2_s1: 0.0543
Epoch 4/5
1563/1563 [==============================] - 160s 102ms/step - s1_acc: 0.5249 - s2_acc: 0.4141 - student_loss_1: 1.1996 - 

[0.5842615365982056,
 0.48266154527664185,
 1.0289678573608398,
 1.8359229564666748]

In [ ]:
# Initialize and compile distiller

# Teach S3 from S1, S2 from T
distiller2 = DistillerT3S1(student1=student1, student2=student2, student3=student3, teacher=teacher)
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller2.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller2.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller2.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 321s 202ms/step - s3_acc: 0.3988 - student_loss_3: 1.4051 - distillation_loss_t_s3: 0.0096 - distillation_loss_s1_s3: 0.0124 - distillation_loss_s2_s3: 0.0111
Epoch 2/5
1563/1563 [==============================] - 316s 202ms/step - s3_acc: 0.5524 - student_loss_3: 0.9278 - distillation_loss_t_s3: 0.0126 - distillation_loss_s1_s3: 0.0105 - distillation_loss_s2_s3: 0.0098
Epoch 3/5
1563/1563 [==============================] - 316s 202ms/step - s3_acc: 0.6245 - student_loss_3: 0.7067 - distillation_loss_t_s3: 0.0139 - distillation_loss_s1_s3: 0.0103 - distillation_loss_s2_s3: 0.0101
Epoch 4/5
1563/1563 [==============================] - 315s 202ms/step - s3_acc: 0.6746 - student_loss_3: 0.5693 - distillation_loss_t_s3: 0.0144 - distillation_loss_s1_s3: 0.0104 - distillation_loss_s2_s3: 0.0104
Epoch 5/5
313/313 [==============================] - 8s 24ms/step - s3_acc: 0.7277 - student_loss_3: 0.6308


[0.7289268970489502, 0.6689102053642273]

In [ ]:
student1.save('/saved_models/als_1_s1')
student2.save('/saved_models/als_1_s2')
student3.save('/saved_models/als_1_s3')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_1_s1/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_1_s2/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_1_s3/assets


In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/als_1_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_1_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_1_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

313/313 [==============================] - 8s 24ms/step - s1_acc: 0.7951 - s2_acc: 0.0000e+00 - s3_acc: 0.0000e+00 - student_loss_3: 0.6308


[0.791700005531311, 0.0, 0.0, 0.6689102053642273]

In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/als_1_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_1_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_1_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student2_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

313/313 [==============================] - 2s 4ms/step - s1_acc: 0.5766 - s2_acc: 0.0000e+00 - s3_acc: 0.0000e+00 - student_loss_3: 1.2320


[0.5794000029563904, 0.0, 0.0, 1.8359229564666748]

In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/als_1_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_1_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_1_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student1_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

313/313 [==============================] - 5s 14ms/step - s1_acc: 0.6371 - s2_acc: 0.0000e+00 - s3_acc: 0.0000e+00 - student_loss_3: 1.1345


[0.6383000016212463, 0.0, 0.0, 1.0289678573608398]

In [ ]:
# Initialize and compile distiller

# Teach S1 and S3 from T
student1 = get_student1()
student2 = get_student2()
student3 = get_student3()
distiller1 = DistillerT1S2(student1=student1, student2=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')

distiller1.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller1.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller1.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 402s 253ms/step - s1_acc: 0.2883 - s2_acc: 0.3822 - student_loss_1: 102.9858 - student_loss_2: 1.4219 - distillation_loss_t_s1: 0.0209 - distillation_loss_s1_s2: 0.0279 - distillation_loss_t_s2: 0.0093 - distillation_loss_s2_s1: 0.8643
Epoch 2/5
1563/1563 [==============================] - 396s 253ms/step - s1_acc: 0.4064 - s2_acc: 0.5498 - student_loss_1: 3.1422 - student_loss_2: 0.9110 - distillation_loss_t_s1: 0.0217 - distillation_loss_s1_s2: 0.0207 - distillation_loss_t_s2: 0.0120 - distillation_loss_s2_s1: 0.1173
Epoch 3/5
1563/1563 [==============================] - 396s 253ms/step - s1_acc: 0.4697 - s2_acc: 0.6273 - student_loss_1: 2.1666 - student_loss_2: 0.6851 - distillation_loss_t_s1: 0.0186 - distillation_loss_s1_s2: 0.0166 - distillation_loss_t_s2: 0.0131 - distillation_loss_s2_s1: 0.0733
Epoch 4/5
1563/1563 [==============================] - 396s 254ms/step - s1_acc: 0.5179 - s2_acc: 0.6793 - student_loss_1: 1.2940 -

[0.5791769027709961,
 0.7348769307136536,
 0.9004974961280823,
 0.6769096851348877]

In [ ]:
# Initialize and compile distiller

# Teach S2 from S1, S3 from T
distiller2 = DistillerT3S1(student1=student1, student2=student3, student3=student2, teacher=teacher)
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller2.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller2.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller2.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 136s 84ms/step - s3_acc: 0.2349 - student_loss_3: 2.5231 - distillation_loss_t_s3: 0.0329 - distillation_loss_s1_s3: 0.0452 - distillation_loss_s2_s3: 0.0450
Epoch 2/5
1563/1563 [==============================] - 131s 84ms/step - s3_acc: 0.3503 - student_loss_3: 1.5224 - distillation_loss_t_s3: 0.0086 - distillation_loss_s1_s3: 0.0143 - distillation_loss_s2_s3: 0.0146
Epoch 3/5
1563/1563 [==============================] - 131s 84ms/step - s3_acc: 0.4109 - student_loss_3: 1.3427 - distillation_loss_t_s3: 0.0105 - distillation_loss_s1_s3: 0.0134 - distillation_loss_s2_s3: 0.0136
Epoch 4/5
1563/1563 [==============================] - 131s 84ms/step - s3_acc: 0.4535 - student_loss_3: 1.2080 - distillation_loss_t_s3: 0.0120 - distillation_loss_s1_s3: 0.0130 - distillation_loss_s2_s3: 0.0130
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - s3_acc: 0.5037 - student_loss_3: 1.3103


[0.5045422911643982, 1.2138508558273315]

In [ ]:
student1.save('/saved_models/als_2_s1')
student2.save('/saved_models/als_2_s2')
student3.save('/saved_models/als_2_s3')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_2_s1/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_2_s2/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_2_s3/assets


In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/als_2_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_2_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_2_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student1_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

student1_test = tf.keras.models.load_model('/saved_models/als_2_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_2_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_2_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student2_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

student1_test = tf.keras.models.load_model('/saved_models/als_2_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_2_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_2_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

313/313 [==============================] - 8s 24ms/step - s1_acc: 0.8283 - s2_acc: 0.0000e+00 - s3_acc: 0.0000e+00 - student_loss_3: 0.5441


[0.8242999911308289, 0.0, 0.0, 0.6769096851348877]

In [ ]:
# Initialize and compile distiller

# Teach S1 and S3 from T
student1 = get_student1()
student2 = get_student2()
student3 = get_student3()
distiller1 = DistillerT1S2(student1=student2, student2=student3, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')

distiller1.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller1.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller1.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 335s 211ms/step - s1_acc: 0.1474 - s2_acc: 0.4020 - student_loss_1: 2.8572 - student_loss_2: 1.3758 - distillation_loss_t_s1: 0.0288 - distillation_loss_s1_s2: 0.0224 - distillation_loss_t_s2: 0.0091 - distillation_loss_s2_s1: 0.0339
Epoch 2/5
1563/1563 [==============================] - 330s 211ms/step - s1_acc: 0.2131 - s2_acc: 0.5668 - student_loss_1: 1.8355 - student_loss_2: 0.8740 - distillation_loss_t_s1: 0.0045 - distillation_loss_s1_s2: 0.0155 - distillation_loss_t_s2: 0.0119 - distillation_loss_s2_s1: 0.0116
Epoch 3/5
1563/1563 [==============================] - 330s 211ms/step - s1_acc: 0.2860 - s2_acc: 0.6425 - student_loss_1: 1.4734 - student_loss_2: 0.6653 - distillation_loss_t_s1: 0.0079 - distillation_loss_s1_s2: 0.0137 - distillation_loss_t_s2: 0.0130 - distillation_loss_s2_s1: 0.0112
Epoch 4/5
1563/1563 [==============================] - 330s 211ms/step - s1_acc: 0.3440 - s2_acc: 0.6914 - student_loss_1: 1.2640 - s

[0.41504615545272827,
 0.7437769174575806,
 1.272771954536438,
 0.7766015529632568]

In [ ]:
# Initialize and compile distiller

# Teach S2 from S1, S3 from T
distiller2 = DistillerT3S1(student1=student2, student2=student3, student3=student1, teacher=teacher)
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller2.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller2.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller2.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 169s 105ms/step - s3_acc: 0.2893 - student_loss_3: 2.0311 - distillation_loss_t_s3: 0.0190 - distillation_loss_s1_s3: 0.0204 - distillation_loss_s2_s3: 0.0261
Epoch 2/5
1563/1563 [==============================] - 164s 105ms/step - s3_acc: 0.4063 - student_loss_3: 1.6540 - distillation_loss_t_s3: 0.0192 - distillation_loss_s1_s3: 0.0170 - distillation_loss_s2_s3: 0.0215
Epoch 3/5
1563/1563 [==============================] - 164s 105ms/step - s3_acc: 0.4687 - student_loss_3: 1.2517 - distillation_loss_t_s3: 0.0165 - distillation_loss_s1_s3: 0.0129 - distillation_loss_s2_s3: 0.0156
Epoch 4/5
1563/1563 [==============================] - 164s 105ms/step - s3_acc: 0.5182 - student_loss_3: 1.0269 - distillation_loss_t_s3: 0.0151 - distillation_loss_s1_s3: 0.0114 - distillation_loss_s2_s3: 0.0122
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - s3_acc: 0.5791 - student_loss_3: 1.0946


[0.5804115533828735, 0.9869548082351685]

In [ ]:
student1.save('/saved_models/als_3_s1')
student2.save('/saved_models/als_3_s2')
student3.save('/saved_models/als_3_s3')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_3_s1/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_3_s2/assets
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/models/saved_models/als_3_s3/assets


In [ ]:
student1_test = tf.keras.models.load_model('/saved_models/als_3_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_3_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_3_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student1_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

student1_test = tf.keras.models.load_model('/saved_models/als_3_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_3_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_3_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student2_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

student1_test = tf.keras.models.load_model('/saved_models/als_3_s1')
student2_test = tf.keras.models.load_model('/saved_models/als_3_s2')
student3_test = tf.keras.models.load_model('/saved_models/als_3_s3')

# Initialize and compile distiller
distiller_test = DistillerT3S1(student1=student1_test, student2=student2_test, student3=student3_test, teacher=teacher)
metric_student1 = keras.metrics.SparseCategoricalAccuracy(name='s1_acc')
metric_student2 = keras.metrics.SparseCategoricalAccuracy(name='s2_acc')
metric_student3 = keras.metrics.SparseCategoricalAccuracy(name='s3_acc')

distiller_test.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[metric_student1, metric_student2, metric_student3],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Evaluate student on test dataset
distiller_test.evaluate(x_test, y_test)

313/313 [==============================] - 8s 24ms/step - s1_acc: 0.8224 - s2_acc: 0.0000e+00 - s3_acc: 0.0000e+00 - student_loss_3: 0.5775


[0.8192999958992004, 0.0, 0.0, 0.7766015529632568]